In [34]:
import pandas as pd
import re

In [35]:
tolls = pd.read_csv('csv_files/tolls.csv')
tolls.drop_duplicates()
tolls.head()

,EndZoneID,StartZoneID,StartDateTime,EndDateTime,TollRate,Direction
0,3130,3100,2019-05-17T21:24:00Z,2019-05-17T21:30:00Z,0.0,1100
1,3120,3100,2019-05-17T21:24:00Z,2019-05-17T21:30:00Z,0.0,1100
2,3110,3100,2019-05-17T21:24:00Z,2019-05-17T21:30:00Z,0.0,1100
3,3100,3100,2019-05-17T21:24:00Z,2019-05-17T21:30:00Z,0.0,1100
4,3130,3100,2019-05-17T21:18:00Z,2019-05-17T21:24:00Z,0.0,1100


In [36]:
#pd.set_option('display.max_rows', 200)
tolls['StartTime'] = tolls['StartDateTime'].apply(lambda x: int(re.sub('\D', '', x)))
tolls['EndTime'] = tolls['EndDateTime'].apply(lambda x: int(re.sub('\D', '', x)))
tolls = tolls.sort_values(by=['EndDateTime'], ascending=True)

tolls.head()

,EndZoneID,StartZoneID,StartDateTime,EndDateTime,TollRate,Direction,StartTime,EndTime
1940034,3120,3100,2018-06-07T15:18:00Z,2018-06-07T15:24:00Z,0.0,1100,20180607151800,20180607152400
1940033,3130,3100,2018-06-07T15:18:00Z,2018-06-07T15:24:00Z,0.0,1100,20180607151800,20180607152400
1940289,3230,3210,2018-06-07T15:18:00Z,2018-06-07T15:24:00Z,0.0,1200,20180607151800,20180607152400
1940035,3110,3100,2018-06-07T15:18:00Z,2018-06-07T15:24:00Z,0.0,1100,20180607151800,20180607152400
1940036,3100,3100,2018-06-07T15:18:00Z,2018-06-07T15:24:00Z,0.0,1100,20180607151800,20180607152400


In [37]:
tolls['StartDateTime'] = pd.to_datetime(tolls['StartDateTime'], format='%Y-%m-%dT%H:%M:%SZ', utc=True)
tolls['EndDateTime'] = pd.to_datetime(tolls['EndDateTime'], format='%Y-%m-%dT%H:%M:%SZ', utc=True)
tolls['Direction'] = tolls['Direction'].map({1100: 'E', 1200: 'W'})
tolls.head()

,EndZoneID,StartZoneID,StartDateTime,EndDateTime,TollRate,Direction,StartTime,EndTime
1940034,3120,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,E,20180607151800,20180607152400
1940033,3130,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,E,20180607151800,20180607152400
1940289,3230,3210,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,W,20180607151800,20180607152400
1940035,3110,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,E,20180607151800,20180607152400
1940036,3100,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,E,20180607151800,20180607152400


In [38]:
def toll_time_conversion(x):
    year = x[0:4]
    month = x[5:7]
    day = x[8:10]
    hour = x[11:13]
    minute = x[14:16]
    second = x[17:19]
    time = year + '/' + month + '/' + day + '/' + hour + '/' + minute + '/' + second
    return time

In [39]:
def weather_time_conversion(x):
    year = x[6:10]
    day = x[:2]
    month = x[3:5]
    hour = x[11:13]
    minute = x[14:]
    return int(year + month + day + hour + minute + '00')
    

In [40]:
weather = pd.read_csv('csv_files/weather.csv')
weather = weather.reindex(index=weather.index[::-1])
weather['Time'] = weather['Time'].apply(lambda x: weather_time_conversion(x))

weather.head()

,Time,Temperature,Pressure,Humidity,Wind Direction,Wind Speed,Weather Desc,Cloud Cover,Visibility
13045,20180607005200,19.0,761.9,64.0,Wind blowing from the north-northwest,3.0,NaN,"Scattered clouds (40-50%) 1320 m, overcast (10...",16.0
13044,20180607015200,18.0,761.9,64.0,Wind blowing from the north-northwest,3.0,NaN,Broken clouds (60-90%) 2100 m,16.0
13043,20180607025200,18.0,762.1,64.0,Wind blowing from the north-northwest,2.0,NaN,Scattered clouds (40-50%) 2100 m,16.0
13042,20180607035200,17.0,762.4,73.0,Wind blowing from the north-northwest,3.0,NaN,Scattered clouds (40-50%) 1800 m,16.0
13041,20180607045200,17.0,762.9,73.0,Wind blowing from the north-northwest,3.0,NaN,Broken clouds (60-90%) 1950 m,16.0


In [41]:
temps = []
pressures = []
humidities = []
wind_directions = []
wind_speeds = []
weather_descs = []
cloud_covers = []
visibilities = []
temp = 0.0
pressure = 0.0
humidity = 0.0
wind_direction = ''
wind_speed = 0.0
weather_desc = ''
cloud_cover = ''
visibility = 0

update = True
cur_weather_row = 0
for index, row in tolls.iterrows():
    toll_time = row['StartTime']
    weather_time = weather.iloc[cur_weather_row]['Time']
    while toll_time > weather_time + 10000:
        cur_weather_row += 1
        weather_time = weather.iloc[cur_weather_row]['Time']
        update = True
    if update:
        weather_row = weather.iloc[cur_weather_row]
        temp = weather_row['Temperature']
        pressure = weather_row['Pressure']
        humidity = weather_row['Humidity']
        wind_direction = weather_row['Wind Direction']
        wind_speed = weather_row['Wind Speed']
        weather_desc = weather_row['Weather Desc']
        cloud_cover = weather_row['Cloud Cover']
        visibility = weather_row['Visibility']
        update = False
        
    temps.append(temp)
    pressures.append(pressure)
    humidities.append(humidity)
    wind_directions.append(wind_direction)
    wind_speeds.append(wind_speed)
    weather_descs.append(weather_desc)
    cloud_covers.append(cloud_cover)
    visibilities.append(visibility)

In [42]:
tolls['Temperature'] = temps
tolls['Pressure'] = pressures
tolls['Humidity'] = humidities
tolls['Wind Direction'] = wind_directions
tolls['Wind Speed'] = wind_speeds
tolls['Weather Desc'] = weather_descs
tolls['Cloud Cover'] = cloud_covers
tolls['Visibility'] = visibilities

In [43]:
tolls

,EndZoneID,StartZoneID,StartDateTime,EndDateTime,TollRate,Direction,StartTime,EndTime,Temperature,Pressure,Humidity,Wind Direction,Wind Speed,Weather Desc,Cloud Cover,Visibility
1940034,3120,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,E,20180607151800,20180607152400,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940033,3130,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,E,20180607151800,20180607152400,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940289,3230,3210,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,W,20180607151800,20180607152400,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940035,3110,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,E,20180607151800,20180607152400,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940036,3100,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,E,20180607151800,20180607152400,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940153,3130,3130,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,E,20180607151800,20180607152400,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940134,3120,3120,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,E,20180607151800,20180607152400,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940133,3130,3120,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,E,20180607151800,20180607152400,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940229,3230,3200,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,W,20180607151800,20180607152400,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940349,3230,3230,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,W,20180607151800,20180607152400,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0


In [44]:
tolls = tolls.drop(columns=['StartTime', 'EndTime'])


In [47]:
pd.set_option('display.max_rows', 2000)
tolls

,EndZoneID,StartZoneID,StartDateTime,EndDateTime,TollRate,Direction,Temperature,Pressure,Humidity,Wind Direction,Wind Speed,Weather Desc,Cloud Cover,Visibility
1940034,3120,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.00,E,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940033,3130,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.00,E,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940289,3230,3210,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.00,W,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940035,3110,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.00,E,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940036,3100,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.00,E,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940153,3130,3130,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.00,E,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940134,3120,3120,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.00,E,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940133,3130,3120,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.00,E,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940229,3230,3200,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.00,W,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1940349,3230,3230,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.00,W,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0


In [48]:
tolls.to_csv('./csv_files/toll_weather.csv')

In [33]:
tolls.shape

(2281425, 6)